In [ ]:
from collections import defaultdict, Counter
import json
import string
from pathlib import Path
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statistics
import csv
from matplotlib import pyplot as plt
import seaborn
import operator

current_path = str(Path().absolute())
link_to_data = current_path + "/Kuala_Lumpur.json"

with open(link_to_data) as f:
	data = json.loads(f.read())

In [ ]:
# Final and initial score array 
checkout_array = list()
checkout_hotel_details_array = list()
checkout_hotel_comment_rating=list()
difference =[]


for key, value in data.items():
    sums = 0
    temp_list = list()   
    if 'reviews' in value:
        for name, review in value['reviews'].items():
                current_review=review['review_text']
                # Converts uppercase letters to lowercase
                current_review = current_review.lower()
                # Removes special characters from the review comments 
                review_comments = current_review.translate(str.maketrans('', '', string.punctuation))
                # Creates tokenize words 
                tokenized_reviews = word_tokenize(review_comments,"english")
                #Removing the stop words from the comments 
                final_review_comments = list() 
                # Loop over the tokenized reviews, and append the non stop words 
                for word in tokenized_reviews:
                        if word not in stopwords.words('english'):
                                final_review_comments.append(word)
                # Function to count the sentiments of the given reviews for the hotel        
                def analyse_sentiment(sentiment_text):
                        score = SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
                        negative = score['neg']
                        positive = score['pos']
                        neutral = score['neu']
                        compound = score['compound']
                        #print(score)
                        return compound

                total = analyse_sentiment(review_comments)
                #         print(abs(total))
                #         sums += abs(total)
                        # Append the compound
                temp_list.append(abs(total)*10)
                checkout_hotel_comment_rating.append([review['rating']])
                
         
        #checkout_hotel_details_array.append([temp_list])
        break
        
#difference=checkout_hotel_details_array-checkout_hotel_comment_rating
#difference=list(map(operator.sub,checkout_hotel_details_array,checkout_hotel_comment_rating))


sentiment_ratings = np.array(checkout_hotel_comment_rating)
original_ratings = np.array(temp_list)
checkout_hotel_comment_rating=list(np.concatenate(checkout_hotel_comment_rating).flat)

difference=list(map(operator.sub,original_ratings,checkout_hotel_comment_rating))
difference =  [abs(ele) for ele in difference]
print( checkout_hotel_comment_rating)
print('\n\n')
#printing the comments

print(original_ratings)

plt.style.use('seaborn')
plt.scatter(original_ratings,checkout_hotel_comment_rating,s=90,c=difference,cmap='Blues',alpha=0.75,linewidth=1, edgecolor='blue')
plt.ylabel('Original Rating')
plt.xlabel('Sentiment Rating')
cbar=plt.colorbar()
cbar.set_label('Difference between the 2 ratings')
plt.show()
